In [15]:
#import necessary libraries
import numpy as np
import pandas as pd

In [ ]:
#define constants
m_p=938.27231 #MeV
m_n=939.56563 #MeV
t_lab=rxn_observable['energy'] #Mev
k_p=np.sqrt(((m_p**2)*(t_lab)*(t_lab+2*m_n))/(((m_p+m_n)**2)+(2*t_lab*m_p))) #fm^-1
k_n=np.sqrt((1/2)*m_p*t_lab) #fm^-1
hbar_c=197.327053 #MeVfm

In [14]:
np_total=pd.read_csv('CSV_files/np.dat.csv')
pp_total=pd.read_csv('CSV_files/pp.dat.csv')
pp_total.rename(columns={' scattering_angle': 'scattering_angle', ' experimental_value': 'experimental_value', ' statistical_error': 'statistical_error', ' reaction_type': 'reaction_type', ' observable': 'observable', ' systematic_error': 'systematic_error', ' normalization': 'normalization'}, inplace=True)
np_total.rename(columns={' scattering_angle': 'scattering_angle', ' experimental_value': 'experimental_value', ' statistical_error': 'statistical_error', ' reaction_type': 'reaction_type', ' observable': 'observable', ' systematic_error': 'systematic_error', ' normalization': 'normalization'}, inplace=True)

In [2]:
# read separated files
#np files
np_sgt=pd.read_csv('CSV_files/np.sgt.csv')
np_sgtl=pd.read_csv('CSV_files/np.sgtl.csv')
np_sgtr=pd.read_csv('CSV_files/np.sgtr.csv')
np_sgtt=pd.read_csv('CSV_files/np.sgtt.csv')

#pp files
pp_sgt=pd.read_csv('CSV_files/pp.sgt.csv')
pp_sgte=pd.read_csv('CSV_files/pp.sgte.csv')
pp_sgtl=pd.read_csv('CSV_files/pp.sgtl.csv')
pp_sgtr=pd.read_csv('CSV_files/pp.sgtr.csv')
pp_sgtt=pd.read_csv('CSV_files/pp.sgtt.csv')

In [3]:
#remove uneccesary columns
#total files
pp_total = pp_total.drop(columns=['statistical_error', 'systematic_error', 'normalization'])
np_total = np_total.drop(columns=['statistical_error', 'systematic_error', 'normalization'])
#np files
np_sgt = np_sgt.drop(columns=['statistical_error', 'systematic_error', 'normalization'])
np_sgtl = np_sgtl.drop(columns=['statistical_error', 'systematic_error', 'normalization'])
np_sgtr = np_sgtr.drop(columns=['statistical_error', 'systematic_error', 'normalization'])
np_sgtt = np_sgtt.drop(columns=['statistical_error', 'systematic_error', 'normalization'])
#pp files
pp_sgt = pp_sgt.drop(columns=['statistical_error', 'systematic_error', 'normalization'])
pp_sgte = pp_sgte.drop(columns=['statistical_error', 'systematic_error', 'normalization'])
pp_sgtl = pp_sgtl.drop(columns=['statistical_error', 'systematic_error', 'normalization'])
pp_sgtr = pp_sgtr.drop(columns=['statistical_error', 'systematic_error', 'normalization'])
pp_sgtt = pp_sgtt.drop(columns=['statistical_error', 'systematic_error', 'normalization'])

### Generate Plots

In [5]:
import matplotlib.pyplot as plt
import matplotlib.style as style
style.use('fivethirtyeight')

def generate_plot(rxn_observable, reac_observ):
    #define values to be plotted
    energy=rxn_observable['energy']
    value=rxn_observable['experimental_value']
    angle=rxn_observable['scattering_angle']
    #figure specifications
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(nrows=2, ncols=2, figsize = (15,10))
    fig.autofmt_xdate(rotation = 45)
    fig.suptitle('Reaction_Observable: {}'.format(reac_observ), fontsize=24)
    #plot 1
    ax1.plot(energy, value, 'ro', alpha=0.5)
    ax1.set_xlabel('Energy'); ax1.set_ylabel('Experimental Value'); ax1.set_title('Experimental Value vs. Energy')
    #plot 2
    ax2.plot(angle, value, 'bo', alpha=0.5)
    ax2.set_xlabel('Scattering Angle'); ax2.set_ylabel('Experimental Value'); ax2.set_title('Experimental Value vs. Scattering Angle')
    #plot 3
    ax3.plot(angle, energy, 'go', alpha=0.5)
    ax3.set_xlabel('Scattering Angle'); ax3.set_ylabel('Energy'); ax3.set_title('Scattering Angle vs. Energy')
    #plot 4
    ax4.plot(energy, angle, 'mo', alpha=0.5)
    ax4.set_xlabel('Energy'); ax4.set_ylabel('Scattering Angle'); ax4.set_title('Energy vs. Scattering Angle')

In [6]:
## EVERYTHING THAT NEEDS CHANGED SHOULD BE IN HERE

#define rxn and observable for this round of calculations
rxn_observable=np_sgt
reac_observ='NP_SGT'

#define number of random states
number_of_random_states=10

#define intervals of test size
intervals=19 # 1:50%, 3:25%, 4:20%, 9:10%, 19:5%, 24:4%, 49:2%, 99:1%

In [7]:
# One-hot encode categorical features
rxn_observable = pd.get_dummies(rxn_observable)
# Labels are the values we want to predict
labels = np.array(rxn_observable['experimental_value'])
# Remove the labels from the features
# axis 1 refers to the columns
features = rxn_observable.drop('experimental_value', axis = 1)
# Saving feature names for later use
feature_list = list(features.columns)

# Convert to numpy array
features = np.array(features)

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import numpy as np

def split_and_score(size, number_of_random_states):
    test=np.zeros(number_of_random_states)
    train=np.zeros(number_of_random_states)
    for j in range(number_of_random_states):
        #split
        train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size=size)
        #replace any test_labels=0 with small number to prevent dividing by 0
        #for index, value in enumerate(test_labels):
         #   if (value == 0):
          #      value=1e-3
           #     test_labels=np.where(test_labels==0, value, test_labels)
        #rf.train
        rf = RandomForestRegressor(n_estimators=1000)
        rf.fit(train_features, train_labels);
        predictions = rf.predict(test_features)
        rf_new = RandomForestRegressor(n_estimators = 100, criterion = 'mse', max_depth = None, min_samples_split = 2, min_samples_leaf = 1)
        importances = list(rf.feature_importances_)
        feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]
        feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
        #score
        test[j]=rf.score(test_features,test_labels)
        train[j]=rf.score(train_features,train_labels)
    test_ave=np.mean(test)
    train_ave=np.mean(train)
    test_var=np.sqrt(np.var(test))
    train_var=np.sqrt(np.var(train))
    return test_ave, test_var, train_ave, train_var

In [ ]:
from matplotlib.pyplot import figure

test_mean=np.zeros(intervals) 
train_mean=np.zeros(intervals) 
test_variance=np.zeros(intervals) 
train_variance=np.zeros(intervals) 
for i in range(intervals): 
    #define size
    size=(i+1)/(intervals+1)
    #call split function
    test, test_v, train, train_v = split_and_score(size, number_of_random_states)
    #add to arrays
    test_mean[i]=test
    train_mean[i]=train
    test_variance[i]=test_v
    train_variance[i]=train_v
print("Average Testing Score:", test_mean)
print('Testing Variance', test_variance)
print('')
print('Average Training Score', train_mean)
print('Training Variance', train_variance)
print('')

#plot 

#x-axis
if intervals==1:
    sample_size=[50] #50% intervals
elif intervals==3:
    sample_size=list(range(25,76,25)) #25% intervals
elif intervals==4:
    sample_size=list(range(20,81,20)) #20% intervals 
elif intervals==9:
    sample_size=list(range(10,91,10)) #10% intervals
elif intervals==19:
    sample_size=list(range(5,96,5)) #5% intervals
elif intervals==24:
    sample_size=list(range(4,97,4)) #4% intervals
elif intervals==49:
    sample_size=list(range(2,99,2)) #2% intervals
elif intervals==99:
    sample_size=list(range(1,100,1)) #1% intervals

#figure
figure(figsize=(15, 5))
ax=plt.subplot(111)
plt.xlabel('Test Size (%)')
plt.ylabel('Score')
plt.title("Reaction_Observable: {}".format(reac_observ))
plt.xlim(left=0, right=100)
ax.scatter(sample_size, test_mean, s=100, label='Average Testing Score')
ax.scatter(sample_size, train_mean, s=100, label='Average Training Score')
ax.errorbar(sample_size, test_mean, yerr=test_variance, linestyle='none', capsize=10, elinewidth=2)
ax.errorbar(sample_size, train_mean, yerr=train_variance, linestyle='none', capsize=10, elinewidth=2)
ax.legend(loc='best')
plt.show

generate_plot(rxn_observable, reac_observ)